In [1]:
import pandas as pd
import numpy as np
import re
import datetime

In [86]:
cities=['newyork', 'boston', 'losangeles', 'miami', 'seattle', 'saltlakecity', 'houston', 'dallas', 'memphis', 
'atlanta', 'austin', 'cleveland', 'charlotte', 'longisland', 'newjersey', 'lasvegas', 'minneapolis', 'chicago',
'orlando', 'sfbay']
index=0
for city in cities:
    if index == 0:
        df = pd.read_csv('data_craigslist'+str(city)+'.csv').drop(columns=['Unnamed: 0'])
        index += 1
    else:
        df = pd.concat([df, pd.read_csv('data_craigslist'+str(city)+'.csv').drop(columns=['Unnamed: 0'])], axis=0)

df

,Price,Location,URL,Date,Title,part type,num image,text,bicycle frame material,bicycle type,...,frame size,handlebar type,make / manufacturer,suspension,wheel size,model name / number,size / dimensions,serial number,paint color,year manufactured
0,20.0,newyork,https://newyork.craigslist.org/wch/bop/d/new-r...,2022-04-18 18:17,Bike parts for many bikes,other,17,"Mavic, Nomande Motocane, Magna Imposter, Next ...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,250.0,newyork,https://newyork.craigslist.org/brk/bik/d/new-y...,2022-04-18 13:40,Bicycles for Sale Saturday and Sunday road bik...,NaN,7,Bike Sale * bikes for sale many styles and s...,other/unknown,road,...,all sizes,other/unknown,trek schwinn raleigh giant,other/unknown,other/unknown,NaN,NaN,NaN,NaN,NaN
2,10000.0,newyork,https://newyork.craigslist.org/lgi/mpo/d/saint...,2022-04-18 12:36,Buying all dirt bikes quads and jetskis!,NaN,0,Buying all dirt bikes quads and atvs in any co...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,15.0,newyork,https://newyork.craigslist.org/mnh/bop/d/new-y...,2022-04-18 12:22,Freewheel Sprocket Cog 1/8” 16T Single Speed R...,NaN,3,Specification: Brand new and high quality Mate...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Freewheel,NaN,NaN,NaN,NaN
4,750.0,newyork,https://newyork.craigslist.org/brk/bik/d/brook...,2022-04-18 12:10,Fuji sportif 2.3 racing bikes like new. 54cm,NaN,6,In like new condition fuji sportif 2.3 Aluminu...,aluminum,road,...,54cm medium,NaN,NaN,NaN,700C,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,300.0,sfbay,https://sfbay.craigslist.org/sby/bik/d/los-gat...,2022-04-15 15:39,2 Bikes For Sale,NaN,2,2 bikes for sale $300 each Message for more qu...,other/unknown,mountain,...,medium,other/unknown,NaN,other/unknown,other/unknown,NaN,NaN,NaN,NaN,NaN
90,600.0,sfbay,https://sfbay.craigslist.org/sby/bik/d/san-jos...,2022-04-15 15:35,STATE 4130 All Road size large 5-9 to 6ft,NaN,0,state 4130 all road bike single speed 650b car...,steel,gravel,...,large,NaN,state bikes,NaN,650B,NaN,NaN,NaN,NaN,NaN
91,240.0,sfbay,https://sfbay.craigslist.org/scz/mpo/d/pebble-...,2022-04-15 15:11,Leather Motorcycle Jacket,NaN,8,"Size large, I wear a medium, fits me fine, I h...",NaN,NaN,...,NaN,NaN,"Flying Bikes , New York",NaN,NaN,NaN,Large,NaN,NaN,NaN
92,60.0,sfbay,https://sfbay.craigslist.org/sby/bop/d/san-jos...,2022-04-15 14:43,Kids bikes,NaN,4,"Kids bikes, Specialized, Redline, Jamis. $60 e...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
df.columns

Index(['Price', 'Location', 'URL', 'Date', 'Title', 'part type', 'num image',
       'text', 'bicycle frame material', 'bicycle type', 'brake type',
       'condition', 'electric assist', 'frame size', 'handlebar type',
       'make / manufacturer', 'suspension', 'wheel size',
       'model name / number', 'size / dimensions', 'serial number',
       'paint color', 'year manufactured'],
      dtype='object')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1876 entries, 0 to 93
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Price                   1876 non-null   float64
 1   Location                1876 non-null   object 
 2   URL                     1876 non-null   object 
 3   Date                    1876 non-null   object 
 4   Title                   1876 non-null   object 
 5   part type               52 non-null     object 
 6   num image               1876 non-null   int64  
 7   text                    1296 non-null   object 
 8   bicycle frame material  620 non-null    object 
 9   bicycle type            620 non-null    object 
 10  brake type              189 non-null    object 
 11  condition               618 non-null    object 
 12  electric assist         131 non-null    object 
 13  frame size              591 non-null    object 
 14  handlebar type          144 non-null    ob

In [48]:
print(df['bicycle frame material'].unique())

['unknown' 'aluminum' 'steel' 'titanium' 'alloy' 'carbon fiber'
 'composite']


In [47]:
def fix_bfm(x):
    if pd.isna(x) or x.strip() == 'other/unknown':
        return 'unknown'
    else:
        return x.strip()

df['bicycle frame material'] = df['bicycle frame material'].apply(fix_bfm)

In [50]:
df['bicycle type'].unique()

array(['unknown', 'road', 'bmx', 'mountain', 'cargo/pedicab',
       'hybrid/comfort', 'gravel', 'kids', 'track', 'cruiser', 'folding',
       'recumbent/trike', 'unicycle', 'tandem'], dtype=object)

In [49]:
def fix_bt(x):
    if pd.isna(x) or x.strip() == 'other':
        return 'unknown'
    else:
        return x.strip()

df['bicycle type'] = df['bicycle type'].apply(fix_bt)

In [52]:
df['frame size'].unique()

array(['unknown', 'large', 'medium', 'small'], dtype=object)

In [51]:
def fix_fs(x):
    if pd.isna(x):
        return 'unknown'
    numbers = re.findall(r'\d+', x)
    if len(numbers) == 0:
        if 'medium' in x.lower():
            return 'medium'
        elif 'large' in x.lower():
            return 'large'
        elif 'small' in x.lower():
            return 'small'
        else:
            return 'unknown'
    else:
        if '"' or "'" or 'inches' or 'inch' or 'in' in x.lower(): #it's in inches
            size = float(numbers[0]) # assume it's the first number
            if size < 16:
                return 'small'
            if size > 19:
                return 'large'
            else:
                return 'medium'
        else: #cm
            size = float(numbers[0])
            if size < 53:
                return 'small'
            if size > 56:
                return 'large'
            else:
                return 'medium'

df['frame size'] = df['frame size'].apply(fix_fs)

In [54]:
df['suspension'].unique()

array([0, 1], dtype=int64)

In [53]:
def fix_s(x):
    if pd.isna(x):
        return 0
    if 'fork' in x.lower():
        return 1
    else:
        return 0

df['suspension'] = df['suspension'].apply(fix_s)

In [68]:
df['wheel size'].unique()

array(['unknown', 29, 26, 20, 27, 24, 12, 16, 18, 25, 14, 10, 28],
      dtype=object)

In [66]:
def fix_ws(x):
    if pd.isna(x):
        return 'unknown'
    if x.strip() == '700C':
        return 29
    if '650' in x.strip():
        return 26
    numbers = re.findall(r'\d+', x)
    if len(numbers)==0:
        return 'unknown'
    else:
        return int(numbers[0])

df['wheel size'] = df['wheel size'].apply(fix_ws)

In [87]:
df['condition'].unique()

array([nan, ' excellent', ' good', ' fair', ' like new', ' new',
       ' salvage'], dtype=object)

In [88]:
def fix_c(x):
    if pd.isna(x):
        return 'unknown'
    elif 'good' in x:
        return 'good'
    elif 'fair' in x:
        return 'fair'
    elif 'salvage' in x.strip():
        return 'poor'
    elif 'new' or 'excellent' in x:
        return 'excellent'
    else:
        return 'unknown'

df['condition'] = df['condition'].apply(fix_c)

In [89]:
df['condition'].unique()

array(['unknown', 'excellent', 'good'], dtype=object)

In [19]:
df['electric assist'].unique()

array([nan, ' pedal assist', ' none', ' throttle'], dtype=object)

In [20]:
def fix_ea(x):
    if pd.isna(x) or 'none' in x:
        return 0
    else:
        return 1

df['electric assist'] = df['electric assist'].apply(fix_ea)

In [21]:
df['make / manufacturer'].unique()

array([nan, ' Velosolex', ' Mongoose', ' trek giant Bianchi vitus',
       ' raleigh trek schwinn giant', ' Saris', ' Trek,Cannondale,GT',
       ' Haro GT Dyno Mongoose SE', ' trek giant diamondback',
       ' trek schwinn raleigh giant', ' TREK', ' Zizzo', ' SE',
       ' trek giant fuji schwinn raleigh', ' schwin, Huffy', ' Coaster',
       ' Shimano', ' Metakoo', ' SE Bikes', ' GT Bikes', ' NYC Bikes',
       ' fuji trek giat specialized', ' Se Bikes', ' Dynacraft',
       ' Rad Power Bikes', ' trek fuji giant raleigh', ' RAD POWER BIKES',
       ' RAD', ' jamis trek giant schwinn', ' trek fuji panasonic',
       ' trek giant gary fisher', ' Cyberbike', ' James', ' SportRack',
       ' Kazam', ' Honda', ' various', " I don't know",
       ' Schwinn Spin Bikes', ' Greenstar Bikes'], dtype=object)

In [22]:
def fix_mm(x):
    if pd.isna(x) or 'various' in x.lower() or x=="I don't know": 
        return 'other/unknown'
    if 'trek' in x.lower():
        return 'trek'
    if 'mongoose' in x.lower():
        return 'mongoose'
    if 'se' in x.lower():
        return 'se'
    else:
        return x.strip().lower()

df['make / manufacturer'] = df['make / manufacturer'].apply(fix_mm)

In [23]:
df['model name / number'].unique()

array([nan, ' Hutch', ' Bones', " Matte Trek Black/Blazin' Orange",
       ' Via', ' Fat', ' Stretch Rack', ' 350 Venture', ' BR-R7000',
       ' Trak100', ' Freewheel', ' CG BIG RIPPER',
       ' 2021 Pro Performer 20"', ' The Ultimate New York Bike',
       ' City Grounds Big Flyer', ' Mountain bikes',
       ' Radmini Step Thru 2', ' RadMini4', ' Woody Bike Stand',
       ' Rack Roof Bike Carrier SR4882RP', ' WILDERNESS TRAIL BIKES',
       ' Goldwing 1500', ' various', ' Ninja', ' AC Power', ' EcoCross'],
      dtype=object)

In [24]:
df['brake type'].unique()

array([nan, ' other/unknown', ' u-brakes', ' gyro/bmx',
       ' disc (mechanical)', ' v-brakes', ' caliper', ' disc (hydraulic)',
       ' none'], dtype=object)

In [25]:
def fix_bt(x):
    if pd.isna(x) or 'other/unknown' in x.lower() or 'none' in x.lower():
        return 'other/unknown'
    else:
        return x.strip().lower()

df['brake type'] = df['brake type'].apply(fix_bt)

In [26]:
df['handlebar type'].unique()

array([nan, ' drop', ' other/unknown', ' flat', ' bmx', ' riser'],
      dtype=object)

In [27]:
def fix_ht(x):
    if pd.isna(x) or 'other/unknown' in x:
        return 'other/unknown'
    else:
        return x.strip().lower()

df['handlebar type'] = df['handlebar type'].apply(fix_ht)

In [28]:
df['part type'].unique()

array([nan, ' other', ' racks, fenders, bags', ' brakes', ' saddle'],
      dtype=object)

In [29]:
def fix_pt(x):
    if pd.isna(x) or 'other' in x:
        return 0
    else:
        return 1

df['part type'] = df['part type'].apply(fix_pt)
df = df.where(df['part type']==0).dropna(how='all').drop(columns=['part type'])

In [30]:
df = df.drop(columns=['size / dimensions', 'serial number'])

In [31]:
df.columns = [column.replace(' ', '') for column in df.columns]

In [34]:
#df['days'] = [(datetime.datetime.today()-datetime.datetime.strptime(date_, "%Y-%m-%d %H:%M")).days for date_ in df['Date']]

In [35]:
df.drop_duplicates(subset=None, keep='first', inplace=False).to_csv('preprocessed_data_craigslist.csv')

In [36]:
df.drop_duplicates(subset=None, keep='first', inplace=False)



,Price,Neighborhood,URL,bicycleframematerial,bicycletype,framesize,suspension,wheelsize,numimage,text,condition,electricassist,make/manufacturer,modelname/number,braketype,handlebartype
0,1900.0,"hudson, fl",https://newyork.craigslist.org/brk/bik/d/hudso...,other/unknown,other/unknown,other/unknown,0.0,other/unknown,0.0,NaN,other/unknown,0.0,other/unknown,NaN,other/unknown,other/unknown
1,300.0,fairfield ct,https://newyork.craigslist.org/fct/bik/d/fairf...,other/unknown,other/unknown,other/unknown,0.0,other/unknown,0.0,NaN,other/unknown,0.0,other/unknown,NaN,other/unknown,other/unknown
2,190.0,brooklyn,https://newyork.craigslist.org/brk/bik/d/brook...,other/unknown,other/unknown,other/unknown,0.0,other/unknown,0.0,NaN,other/unknown,0.0,other/unknown,NaN,other/unknown,other/unknown
3,250.0,west village,https://newyork.craigslist.org/mnh/bik/d/new-y...,other/unknown,other/unknown,other/unknown,0.0,other/unknown,0.0,NaN,other/unknown,0.0,other/unknown,NaN,other/unknown,other/unknown
4,800.0,brooklyn,https://newyork.craigslist.org/brk/bik/d/brook...,other/unknown,other/unknown,other/unknown,0.0,other/unknown,0.0,NaN,other/unknown,0.0,other/unknown,NaN,other/unknown,other/unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446,275.0,11231,https://newyork.craigslist.org/brk/bik/d/brook...,other/unknown,hybrid/comfort,other/unknown,0.0,other/unknown,10.0,"I have one in each size, 16"", 18"", and 20"". Th...",excellent,0.0,other/unknown,NaN,other/unknown,other/unknown
447,999.0,financial district,https://newyork.craigslist.org/mnh/bik/d/new-y...,aluminum,bmx,other/unknown,0.0,29,7.0,"The PK Ripper was the raddest, baddest, and mo...",excellent,0.0,other/unknown,NaN,other/unknown,other/unknown
448,3850.0,brooklyn,https://newyork.craigslist.org/brk/bik/d/brook...,steel,gravel,other/unknown,0.0,27,0.0,I ordered this bike back in NOV.2020 with all ...,other/unknown,0.0,other/unknown,NaN,other/unknown,other/unknown
449,125.0,garden city,https://newyork.craigslist.org/lgi/bik/d/new-h...,other/unknown,bmx,medium,0.0,20,9.0,Haro F3 BMX-$175 Mongoose Racer X BMX-$175 T...,other/unknown,0.0,other/unknown,NaN,other/unknown,other/unknown


In [ ]:
df